In [7]:
from lxml import etree
import io

In [23]:
xslt_root = etree.XML('''\
<xsl:stylesheet xmlns:xsl="http://www.w3.org/1999/XSL/Transform" xmlns:xs="http://www.w3.org/2001/XMLSchema" xmlns:saxon="http://saxon.sf.net/" version="1.0">
  
  <xsl:param name="recordSetPath"/>
  <xsl:param name="fileNamePattern" select="'*.xml'"/>
  <xsl:param name="showValues" select="1"/>
  <xsl:param name="showFilename" select="1"/>
  <xsl:param name="showCollectionName" select="1"/>
  <xsl:param name="delimiter" select="','"/>
  <xsl:output method="text"/>
  <xsl:strip-space elements="*"/>
  <xsl:template match="/">
    <xsl:if test="$showCollectionName">
      <xsl:text>Collection</xsl:text>
      <xsl:value-of select="$delimiter"/>
    </xsl:if>
    <xsl:if test="$showFilename">
      <xsl:text>Record</xsl:text>
      <xsl:value-of select="$delimiter"/>
    </xsl:if>
    <xsl:text>XPath</xsl:text>
    <xsl:if test="$showValues">
      <xsl:value-of select="$delimiter"/>
      <xsl:text>Content</xsl:text>
    </xsl:if>
    <xsl:text>&#xa;</xsl:text>
    <xsl:for-each select="tokenize($fileNamePattern, ' ')">
      <xsl:variable name="xmlFilesSelect" select="concat($recordSetPath, '?select=', string(.))"/>
      <xsl:for-each select="collection(iri-to-uri($xmlFilesSelect))">
        <!-- Determine Collection Name using the standard directory structure collection/dialect/xml/metadataRecords.
          This option is only useful in the standard collection analysis scenario.
        -->
        <!--<xsl:variable name="collectionName">
          <xsl:for-each select="tokenize(document-uri(.), '/')">
            <xsl:if test="position() = last() - 3">
              <xsl:value-of select="."/>
            </xsl:if>
          </xsl:for-each>
        </xsl:variable>-->
        <xsl:apply-templates>
          <xsl:with-param name="collectionName" select="tokenize(document-uri(.), '/')[last() - 3]"/>
          <xsl:with-param name="fileName" select="tokenize(document-uri(.), '/')[last()]"/>
        </xsl:apply-templates>
      </xsl:for-each>
    </xsl:for-each>
  </xsl:template>
  <xsl:template match="node()">
    <xsl:param name="fileName"/>
    <xsl:param name="collectionName"></xsl:param>
    
    <xsl:if test="not(normalize-space(text()[1]) = '')">
      <xsl:if test="$showCollectionName">
        <xsl:value-of select="$collectionName"/>
        <xsl:value-of select="$delimiter"/>
      </xsl:if>
      <xsl:if test="$showFilename">
        <xsl:value-of select="$fileName"/>
        <xsl:value-of select="$delimiter"/>
      </xsl:if>
      <xsl:value-of select="replace(saxon:path(), '\[\d*\]', '')"/>
      <xsl:if test="$showValues">
        <xsl:value-of select="$delimiter"/>
        <xsl:text>"</xsl:text>
        <xsl:value-of select="replace(normalize-space(replace(., '\[\d*\]', '')), '&quot;', '')"/>
        <xsl:text>"</xsl:text>
      </xsl:if>
      <xsl:text>&#xa;</xsl:text>
    </xsl:if>
    <xsl:for-each select="@*">
      <xsl:if test="$showCollectionName">
        <xsl:value-of select="$collectionName"/>
        <xsl:value-of select="$delimiter"/>
      </xsl:if>
      <xsl:if test="$showFilename">
        <xsl:value-of select="$fileName"/>
        <xsl:value-of select="$delimiter"/>
      </xsl:if>
      <xsl:value-of select="normalize-space(replace(saxon:path(), '\[\d*\]', ''))"/>
      <xsl:if test="$showValues">
        <xsl:value-of select="$delimiter"/>
        <xsl:text>"</xsl:text>
        <!-- remove double quotes -->
        <xsl:value-of select="normalize-space(replace(., '&quot;',''))"/>
        <xsl:text>"</xsl:text>
      </xsl:if>
      <xsl:text>&#xa;</xsl:text>
    </xsl:for-each>
    <xsl:apply-templates select="*">
      <xsl:with-param name="collectionName" select="$collectionName"/>
      <xsl:with-param name="fileName" select="$fileName"/>
    </xsl:apply-templates>
  </xsl:template>
</xsl:stylesheet>
''')
transform = etree.XSLT(xslt_root)

f = io.StringIO('<dummy/>')
doc = etree.parse(f)
result_tree = transform(doc)

        

XSLTApplyError: Failed to evaluate the 'select' expression.

In [24]:
print(transform.error_log)

<string>:0:0:ERROR:XPATH:XPATH_UNKNOWN_FUNC_ERROR: Unregistered function
<xslt>:26:0:ERROR:XSLT:ERR_OK: runtime error, element 'for-each'
<string>:0:0:ERROR:XSLT:ERR_OK: Failed to evaluate the 'select' expression.


In [25]:
for entry in transform.error_log:
     print('message from line %s, col %s: %s' % (
                entry.line, entry.column, entry.message))
     print('domain: %s (%d)' % (entry.domain_name, entry.domain))
     print('type: %s (%d)' % (entry.type_name, entry.type))
     print('level: %s (%d)' % (entry.level_name, entry.level))
     print('filename: %s' % entry.filename)

message from line 0, col 0: Unregistered function
domain: XPATH (12)
type: XPATH_UNKNOWN_FUNC_ERROR (1209)
level: ERROR (2)
filename: <string>
message from line 26, col 0: runtime error, element 'for-each'
domain: XSLT (22)
type: ERR_OK (0)
level: ERROR (2)
filename: <xslt>
message from line 0, col 0: Failed to evaluate the 'select' expression.
domain: XSLT (22)
type: ERR_OK (0)
level: ERROR (2)
filename: <string>
